# 유튜브 데이터 임베딩 및 학습

이 노트북은 유튜브 데이터를 사용하여 **문장 쌍 임베딩**을 수행하고, 이를 기반으로 유튜브 비디오 카테고리 분류 모델을 학습하는 과정을 다룹니다. 우리는 **KoSimCSE-roberta-multitask** 모델을 활용하여 텍스트 데이터를 임베딩하고, 해당 임베딩을 통해 텍스트 분류 작업을 진행합니다.

## 모델 및 학습 개요
사용하는 모델은 **BM-K/KoSimCSE-roberta-multitask**로, 이 모델은 한국어 문장 임베딩에 최적화되어 있습니다. 다양한 멀티태스크 학습을 통해 여러 자연어 처리 작업에서 우수한 성능을 보여주며, 특히 문장 간의 의미적 유사성을 효과적으로 포착하여, 텍스트 분류나 추론 작업에서 뛰어난 성능을 발휘합니다.

## 데이터 설명
이 프로젝트에서는 유튜브 데이터의 일부 샘플을 사용합니다. 해당 데이터는 **자연어 추론(NLI)** 형식으로 변환되어 있으며, 두 개의 문장 쌍(sentence1, sentence2)과 이들 간의 관계를 나타내는 레이블로 구성됩니다. 

데이터셋은 **train**, **valid**, **test**로 분할되어 있으며, 각 데이터는 `.tsv` 파일로 저장되어 있습니다. 이를 기반으로 분류 모델을 학습하고 성능을 평가합니다.

## 주요 단계:
1. **문장 임베딩**: KoSimCSE-roberta-multitask 모델을 사용해 유튜브 텍스트 데이터를 벡터로 변환.
2. **데이터 전처리**: NLI 형식의 데이터를 로딩하고, 학습에 적합한 형태로 정제.
3. **모델 학습**: 임베딩된 데이터를 바탕으로 유튜브 비디오 카테고리 분류 모델을 학습.
4. **모델 평가**: valid와 test 데이터를 사용하여 모델의 성능을 평가하고 분석.

이 과정을 통해 유튜브 데이터의 문장을 효율적으로 임베딩하고, 이를 기반으로 카테고리 분류 작업을 성공적으로 수행할 수 있습니다.


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
# 하이퍼파라미터 설정
MAX_LEN = 50
BATCH_SIZE = 16
EPOCHS = 3
MODEL_NAME = 'BM-K/KoSimCSE-roberta-multitask'
cache_dir = "../models"
# 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=cache_dir)
model = AutoModel.from_pretrained(MODEL_NAME, cache_dir=cache_dir).to('cuda')  # 모델을 GPU로 이동

In [3]:
# 데이터 로딩
train = pd.read_csv('../data/video_sample_train_nli.csv', delimiter='\t', header=None)  # .tsv 파일 경로
vaild = pd.read_csv('../data/video_sample_vaild_sts.csv', delimiter='\t', header=None)  # .tsv 파일 경로
test = pd.read_csv('../data/video_sample_test_sts.csv', delimiter='\t', header=None)  # .tsv 파일 경로

train.columns = ["sentence1", "sentence2", "label"]
vaild.columns = ["sentence1", "sentence2", "label"]
test.columns = ["sentence1", "sentence2", "label"]

In [4]:
# 데이터셋 클래스 정의
class SentencePairDataset(Dataset):
    def __init__(self, sentences1, sentences2, labels, tokenizer, max_len):
        # 초기화 메서드
        self.sentences1 = sentences1  # 첫 번째 문장 리스트
        self.sentences2 = sentences2  # 두 번째 문장 리스트
        self.labels = labels  # 각 문장 쌍에 대한 레이블
        self.tokenizer = tokenizer  # 사용될 토크나이저
        self.max_len = max_len  # 최대 토큰 길이

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메서드
        return len(self.labels)

    def __getitem__(self, index):
        # 주어진 인덱스에 대한 데이터 항목을 반환하는 메서드
        sentence1 = str(self.sentences1[index])  # 첫 번째 문장
        sentence2 = str(self.sentences2[index])  # 두 번째 문장
        label = self.labels[index]  # 레이블

        # 문장 토큰화
        encoding = self.tokenizer.encode_plus(
            sentence1,  # 첫 번째 문장
            sentence2,  # 두 번째 문장
            add_special_tokens=True,  # 특수 토큰 추가 (예: [CLS], [SEP])
            max_length=self.max_len,  # 최대 길이 설정
            return_token_type_ids=False,  # 토큰 타입 ID 반환 여부
            padding='max_length',  # 최대 길이에 맞춰 패딩
            truncation=True,  # 최대 길이를 초과하는 경우 잘라내기
            return_attention_mask=True,  # 어텐션 마스크 반환 여부
            return_tensors='pt',  # PyTorch 텐서로 반환
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),  # 입력 ID
            'attention_mask': encoding['attention_mask'].flatten(),  # 어텐션 마스크
            'labels': torch.tensor(label, dtype=torch.float)  # 레이블을 float으로 변환
        }


In [5]:
# 데이터셋 및 데이터로더 생성
train_dataset = SentencePairDataset(train["sentence1"], train["sentence2"], train["label"], tokenizer, MAX_LEN)
val_dataset = SentencePairDataset(vaild["sentence1"], vaild["sentence2"], vaild["label"], tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
# 최적화기 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [11]:
# 모델 학습
# 모델을 학습 모드로 설정
model.train()

# 지정한 에포크 수만큼 반복
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')  # 현재 에포크 출력
    for batch in tqdm(train_loader):  # 학습 데이터 로더에서 배치 단위로 반복
        optimizer.zero_grad()  # 기울기 초기화

        # 배치의 데이터를 GPU로 전송
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda').float()  # labels를 float으로 변환

        # 모델의 예측값 계산 (last_hidden_state를 가져옴)
        outputs = model(input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state  # 마지막 히든 스테이트
        cls_embedding = hidden_state[:, 0, :]  # 첫 번째 토큰 (CLS 토큰)의 임베딩

        # 분류를 위한 로짓으로 변환 (선형층을 추가로 학습시키는 방식)
        logits = torch.nn.Linear(cls_embedding.size(-1), 1).to('cuda')(cls_embedding)

        # 손실 계산
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logits.view(-1), labels)
        loss.backward()  # 기울기 계산
        optimizer.step()  # 가중치 업데이트

    print(f'Train Loss: {loss.item()}')  # 에포크마다 손실 출력


Epoch 1/3


100%|██████████| 3/3 [00:01<00:00,  2.75it/s]


Train Loss: 0.7290389537811279
Epoch 2/3


100%|██████████| 3/3 [00:00<00:00,  5.60it/s]


Train Loss: 0.7495900392532349
Epoch 3/3


100%|██████████| 3/3 [00:00<00:00,  6.52it/s]

Train Loss: 0.7552095055580139


In [13]:
# 모델 저장
MODEL_PATH = '../models/ko_simcse_model'
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

('../models/ko_simcse_model\\tokenizer_config.json',
 '../models/ko_simcse_model\\special_tokens_map.json',
 '../models/ko_simcse_model\\vocab.txt',
 '../models/ko_simcse_model\\added_tokens.json',
 '../models/ko_simcse_model\\tokenizer.json')

In [14]:
# 학습된 모델과 토크나이저 불러오기
MODEL_PATH = '../models/ko_simcse_model'
model = AutoModel.from_pretrained(MODEL_PATH).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [18]:
# 테스트 데이터셋 및 데이터로더 생성
MAX_LEN = 50

test_dataset = SentencePairDataset(test['sentence1'], test['sentence2'], test['label'], tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [19]:
# 모델 임베딩 추출
model.eval()  # 평가 모드로 전환

In [23]:
# 테스트 데이터에 대한 임베딩 계산
embeddings = []  # 임베딩을 저장할 리스트 초기화

# 기울기 계산을 하지 않도록 설정
with torch.no_grad():
    # 테스트 데이터 로더에서 배치 단위로 반복
    for batch in test_loader:
        input_ids = batch['input_ids'].to('cuda')  # 배치의 input_ids를 GPU로 전송
        attention_mask = batch['attention_mask'].to('cuda')  # 배치의 attention_mask를 GPU로 전송

        # 모델의 출력 (임베딩) 계산
        outputs = model(input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # 첫 번째 토큰 (CLS)의 임베딩 추출

        # CPU로 이동하여 리스트에 저장
        embeddings.append(cls_embedding.cpu().numpy())

# 최종 임베딩 리스트를 하나의 배열로 변환
embeddings = np.concatenate(embeddings, axis=0)  # 리스트 내 모든 배열을 하나의 배열로 결합
embeddings  # 최종 임베딩 출력


array([[-0.45955634, -0.42210102,  0.12629461, ..., -0.910375  ,
        -0.62430143,  0.1694361 ],
       [-0.55795133, -0.27427074,  0.17910959, ..., -0.79411215,
        -0.8216931 , -0.00333627],
       [-0.82763994, -0.36810002, -0.03334512, ..., -0.64958715,
        -0.42699406,  0.26194212],
       ...,
       [-0.45411685, -0.49418557,  0.10794151, ..., -0.69248176,
        -0.4309753 ,  0.06140001],
       [-0.55488634, -0.16833813,  0.2165733 , ..., -0.6129886 ,
        -0.41975313, -0.10078175],
       [-0.47323793, -0.3883717 ,  0.04399573, ..., -0.5406112 ,
        -0.35525244,  0.16355415]], dtype=float32)